# Experimento TCC

## Libs

In [1]:
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NeighborhoodComponentsAnalysis
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, matthews_corrcoef
from tqdm import tqdm
from multiprocessing import cpu_count


import pandas as pd
import numpy as np

In [2]:
from modules.classNCA import NCA
from modules.optimization_funcs import PSO, Gradient

In [3]:
dtypes = {
    'tau1': float, 'tau2': float, 'tau3': float, 'tau4': float,'p1': float, 'p2': float,
    'p3': float, 'p4': float, 'g1': float, 'g2': float, 'g3': float, 'g4': float}

EletricalGrid = pd.read_csv('dataset/Data_for_UCI_named.csv',
                            dtype=dtypes,nrows=180).drop('stab', axis=1)
EletricalGrid.stabf.value_counts()

unstable    114
stable       66
Name: stabf, dtype: int64

In [4]:
X, y = EletricalGrid.drop('stabf',axis=1), EletricalGrid.stabf 

In [5]:
pso_options = {'optimization_func':PSO, 'max_iter':100, 'k':5, 'cpu_count':4}#cpu_count())
gradient_options = {'optimization_func':Gradient, 'max_iter':100, 'k':5,'cpu_count':None}#cpu_count())

In [6]:
def folds_run(X, y, model_class=None, model_options={}):
    kf = KFold(n_splits=5)
    folds_results = {i:[] for i in ['acc','mcc']}
    for (train_index, test_index) in tqdm(kf.split(X)):
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X.loc[train_index,:])
        X_test = scaler.transform(X.loc[test_index,:])
        
        y_train, y_test = y.iloc[train_index].copy(), y.iloc[test_index].copy()
        
        model = model_class(**model_options)
        model.fit(X_train,y_train)
        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test,y_pred)
        mcc = matthews_corrcoef(y_test,y_pred)
        folds_results['acc'].append(acc)
        folds_results['mcc'].append(mcc)
    return folds_results

In [7]:
results = folds_run(X, y, model_class=NCA, model_options=pso_options)
for i in results:
    print(f'{i} mean = {np.array(results[i]).mean()}')

0it [00:00, ?it/s]2023-10-18 10:00:57,117 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}

pyswarms.single.global_best:   0%|                                                                                                                                              |0/100
0it [00:00, ?it/s]


ValueError: zero-dimensional arrays cannot be concatenated

In [ ]:
#results = folds_run(X,y, model_class=NCA, model_options=gradient_options)
#for i in results:
#    print(f'{i} mean = {np.array(results[i]).mean()}') 2:42